In [2]:
# Import SQLAlchemy dependencies
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [3]:
# Import pandas dependencies
import pandas as pd

# Transformation Process

In [4]:
#Uploaded CSV files of all basic Stats of NBA Players on 2016-2017 season
nbastats = "Resources/NBAstats.csv"
NBAdf1 = pd.read_csv(nbastats)
#Choose select columns that are relevant to our research
NBAdf2 = NBAdf1[["Player", "PTS", "MP", "Tm"]]
#Sort the DF by MP or minutes played from greatest to least
NBAppg = NBAdf2.sort_values(by = ["MP"], ascending=False )
#drop any rows with null values
NBAppg1 = NBAppg.dropna(how = "any")
#Show DF
NBAppg1

,Player,PTS,MP,Tm
269,LeBron James,26.4,37.8,CLE
333,Kyle Lowry,22.4,37.4,TOR
312,Zach LaVine,18.9,37.2,MIN
562,Andrew Wiggins,23.6,37.2,MIN
529,Karl-Anthony Towns,25.1,37.0,MIN
...,...,...,...,...
334,John Lucas III,0.4,2.2,MIN
291,Terrence Jones,0.0,2.0,MIL
233,Roy Hibbert,0.7,1.8,DEN
248,Danuel House,0.0,1.0,WAS


In [5]:
#Uploaded CSV file of all advanced stats of NBA players from the 2016-2017 season
player = "Resources/Player_impact_estimation.csv"
impact = pd.read_csv(player)
#Pick select columns that we felt were relevant to our research
impact1 = impact[["PLAYER", "USG%", "MIN", "EFG%", "NETRTG"]]
#Sort the DF by MIN or minutes played from greatest to least
impact2 = impact1.sort_values(by = ["MIN"], ascending = False )
#Change name of columns to match NBAppg1 column names
impact3 = impact2.rename(columns = {"PLAYER": "Player", "USG%": "USG%", "MIN" : "MP" , "EFG%" : "EFG%", "NETRTG" : "NETRTG"})
#Show DF
impact3

,Player,USG%,MP,EFG%,NETRTG
6,LeBron James,29.7,37.8,59.4,7.7
32,Kyle Lowry,24.9,37.4,56.9,8.2
173,Andrew Wiggins,28.8,37.2,48.4,-0.5
174,Zach LaVine,21.8,37.2,54.4,-3.6
14,Jimmy Butler,26.5,37.0,49.2,3.0
...,...,...,...,...,...
113,Brice Johnson,38.5,3.1,28.6,44.9
461,Steve Novak,16.9,2.7,35.7,-1.6
484,Larry Sanders,22.4,2.5,25.0,-7.2
326,John Lucas III,16.6,2.1,25.0,46.9


In [9]:
#do an inner Merge the two DF's together on Player
NBAmerge = pd.merge(NBAppg1,impact3, how = "inner", on = ("Player") )
#Drop Extra MP column since they are identical
NBAmerge1 = NBAmerge.drop(columns = ["MP_y"])
#Rename Columns
NBAmerge2 = NBAmerge1.rename(columns = {"Player": "Player", "PTS":"PTS", "MP_x" : "MP" , "Tm": "Team","USG%": "USG%", "EFG%" : "EFG%", "NETRTG" : "NETRTG"})
#Sort DF by MP or minutes played from greatest to least.
NBAmerge3 = NBAmerge2.sort_values(by = ["MP"], ascending = False)
#Do a loc to find all players that averaged atleast 29.2 minutes a game
NBAmerge4 = NBAmerge3.loc[NBAmerge3["MP"] >= 29.2, :]
#Show DF
NBAmerge4

,Player,PTS,MP,Team,USG%,EFG%,NETRTG
0,LeBron James,26.4,37.8,CLE,29.7,59.4,7.7
1,Kyle Lowry,22.4,37.4,TOR,24.9,56.9,8.2
2,Zach LaVine,18.9,37.2,MIN,21.8,54.4,-3.6
3,Andrew Wiggins,23.6,37.2,MIN,28.8,48.4,-0.5
4,Karl-Anthony Towns,25.1,37.0,MIN,27.4,57.6,-0.9
...,...,...,...,...,...,...,...
99,Tony Snell,8.5,29.2,MIL,12.2,58.8,-1.2
100,Jordan Clarkson,14.7,29.2,LAL,23.2,50.0,-8.9
101,Buddy Hield,15.1,29.1,SAC,21.4,52.3,-5.5
104,Yogi Ferrell,11.3,29.1,DAL,19.6,49.2,-4.1


# Load into DATABASE (sqllite)

In [ ]:
#Create engine & connection to sqlite db
engine = create_engine(f"sqlite:///Resources/nba.sqlite")
conn = engine.connect()

In [ ]:
#Give db a name and copy merged df to the sqlite DB
sqlite_table = "NBA"
NBAmerge4.to_sql(sqlite_table, conn, if_exists='fail')

In [ ]:
#close connection
conn.close()

In [ ]:
#create engine to open sqlite
engine = create_engine(f"sqlite:///Resources/nba.sqlite")

In [ ]:
#create for loop to produce all the data in the NBA db
data = engine.execute('SELECT * FROM NBA')
for record in data:
    print(record)